In [1]:
import pandas as pd
import numpy as np
import torch


In [4]:
biometry = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/outputs/biometry_org.csv')

In [6]:
biometry

,ID,HC,head_ps,AC,abdomen_ps,FL,femur_ps
0,1,20.19,0.096716,17.74,0.110422,4.15,0.096809
1,2,19.19,0.107914,15.63,0.113594,3.59,0.076084
2,3,25.19,0.121011,22.21,0.147919,5.38,0.147919
3,4,23.98,0.129155,21.01,0.156729,4.80,0.115368
4,5,17.10,0.102237,15.87,0.094373,3.20,0.087999
...,...,...,...,...,...,...,...
596,703,31.48,0.194965,30.38,0.225223,6.16,0.180680
597,704,19.71,0.139701,17.72,0.132349,4.01,0.089929
598,705,24.34,0.159498,21.59,0.215154,4.83,0.167642
599,706,22.59,0.217095,21.02,0.217095,4.56,0.217095


In [43]:
femur_biometry = biometry[['ID', 'FL', 'femur_ps']]


In [48]:
femur_biometry['ID'] = femur_biometry['ID'].astype(str)

/tmp/ipykernel_1491735/4102436457.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  femur_biometry['ID'] = femur_biometry['ID'].astype(str)


In [53]:
for i in range(len(femur_biometry)):
    idx = femur_biometry.loc[i, 'ID']
    new_idx = idx + '_3'
    femur_biometry.loc[i, 'ID'] = new_idx
    

In [54]:
femur_biometry

,ID,FL,femur_ps
0,1_3,4.15,0.096809
1,2_3,3.59,0.076084
2,3_3,5.38,0.147919
3,4_3,4.80,0.115368
4,5_3,3.20,0.087999
...,...,...,...
596,703_3,6.16,0.180680
597,704_3,4.01,0.089929
598,705_3,4.83,0.167642
599,706_3,4.56,0.217095


In [69]:
femur_data = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/outputs/femur_data.csv')

In [70]:
len(femur_data['video_x'].unique())

629

In [71]:
femur_merged = pd.merge(femur_data, femur_biometry, left_on='video_x', right_on='ID')

In [72]:
femur_merged = femur_merged.rename(columns={'video_x': 'video'})

In [73]:
femur_merged.drop(['ID', 'score'], axis=1, inplace=True)

In [81]:
femur_merged.to_csv('/data/kpusteln/Fetal-RL/data_preparation/outputs/femur_merged.csv', index=False)

In [75]:
import os
import json
import torch.utils.data as data
import numpy as np
from PIL import Image
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import warnings
import torchvision.transforms as transforms
from torchvision.utils import save_image

warnings.filterwarnings("ignore", "(Possibly )?corrupt EXIF data", UserWarning)


class Fetal(data.Dataset):
    def __init__(self, root, ann_path, transform=None, target_transform=None):

        self.data_path = root
        self.ann_path = ann_path
        self.transform = transform
        self.target_transform = target_transform
        # id & label: https://github.com/google-research/big_transfer/issues/7
        # total: 21843; only 21841 class have images: map 21841->9205; 21842->15027
        self.database = pd.read_csv(self.ann_path)

    def _load_image(self, path):
        try:
            im = Image.open(path)
        except:
            print("ERROR IMG LOADED: ", path)
            random_img = np.random.rand(224, 224, 3) * 255
            im = Image.fromarray(np.uint8(random_img))
        return im

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is class_index of the target class.
        """
        idb = self.database.iloc[index]

        # images
        images = self._load_image(self.data_path  + idb[0] + '.png')
        if self.transform is not None:
            images = self.transform(images)

        # target
        indexes = idb[0]
        target = idb[1]
        #score = idb[2]
        score = 1
        video = idb[2]
        femur_len = idb[3]
        pixel_spacing = idb[4]
        if self.target_transform is not None:
            target = self.target_transform(target)
        #save_image(images[0], '/data/kpusteln/examples' + str(index) + '.png')
        return images, target, indexes, pixel_spacing, video, femur_len

    def __len__(self):
        return len(self.database)


In [138]:
t = transforms.Compose([transforms.ToTensor(),])

In [82]:
fetal = Fetal(root = '/data/kpusteln/fetal/fetal_extracted/', ann_path = './outputs/femur_merged.csv', transform=t)

In [83]:
batch_size = 1

In [84]:
loader = torch.utils.data.DataLoader(
      fetal,
      batch_size=batch_size,
      shuffle=False,
      num_workers=40,
  )

In [133]:
indexes_list = []
height_list = []
width_list = []
labels = []
pixel_spacings = []
videos = []
femur_lens = []
i = 0
px = 0.15
for img, label, index, pixel_spacing, video, femur_len in loader:
    height = img.shape[2]
    width = img.shape[3]
    new_height = height*(pixel_spacing/px)
    new_width = width*(pixel_spacing/px)
    label = label.numpy()
    height_list.extend(new_height.numpy())
    width_list.extend(new_width.numpy())
    indexes_list.extend(index)
    labels.extend(label)
    femur_lens.extend(femur_len.numpy())
    videos.extend(video)
    pixel_spacings.extend(pixel_spacing.numpy())
    i+=1
    if i%1000 == 0:
        print(i)
    

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000


KeyboardInterrupt: 

In [189]:
im1 = Image.open('/data/kpusteln/fetal/fetal_extracted/17_3_191.png') # 975x743

In [ ]:
# 17_3_191,1570.1918066666665,2060.4804999999997,5,17_3,6.05,0.316997

In [181]:
im1.size

(975, 743)

In [188]:
t0 = transforms.Compose([transforms.ToTensor(),])
t1 = transforms.Compose([transforms.Resize((1570, 2060)),
                        transforms.ToTensor()])


t2 = transforms.Compose([transforms.Pad((0, 0, 0, 222), fill = 0, padding_mode = 'constant'),
                         transforms.Resize((224, 224)),
                         transforms.ToTensor()])

In [187]:
save_image(t0(im1), './oryg.png')
save_image(t1(im1), './after_pixel_spacing.png')
save_image(t2(im1), './test2.png')

In [23]:
import numpy as np
import pandas as pd



data = pd.read_csv('./outputs/labels_corrected.csv')
frames_n = data['video'].value_counts()
frames_n = pd.DataFrame({'video': frames_n.index, 'frames_n': frames_n.values})
frames_n.to_csv('/data/kpusteln/Fetal-RL/data_preparation/outputs/frames_n.csv', index=False)


In [17]:
frames_n.values

array([551, 543, 543, ...,  16,  15,  14])

In [18]:
frames_n

529_3    551
418_3    543
418_2    543
58_3     516
66_3     516
        ... 
641_2     22
647_2     20
43_3      16
651_3     15
93_2      14
Name: video, Length: 1516, dtype: int64

In [24]:
import pandas as pd

In [36]:
data_train = pd.read_csv('/data/kpusteln/fetal/fetal_train_final.csv')

In [37]:
videos = pd.DataFrame({'video': data_train['video'].unique()})

In [38]:
videos.to_csv('/data/kpusteln/fetal/videos_train.csv', index=False)